In [19]:
import numpy as np
import pandas as pd
import pickle as pkl
import sklearn
import nltk
import os

from nltk.corpus import stopwords 
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
INPUT_DIR = '../data/input/imperatives/ground_truth/'
OUTPUT_DIR = '../data/'
MODEL_DIR = '../models/'

In [3]:
ip_file = os.path.join(INPUT_DIR, 'imperatives_binary_data.csv')
df_data_raw = pd.read_csv(ip_file)

In [4]:
df_data_raw

,text,label
0,Find a sturdy piece of cardboard in the form o...,1
1,Stand up for yourself,1
2,Fix out priorities together in a meeting a co...,1
3,Make one last snowball for the penguin's head,1
4,Look for the internet venue you will use for y...,1
...,...,...
2319,it's a Finnish documentary but it has all thes...,0
2320,yeah because you took time when you had your ...,0
2321,oh come on you're kidding right.,0
2322,You see.,0


In [5]:
data = df_data_raw['text']
label = df_data_raw['label']

from sklearn.model_selection import train_test_split

data_train, data_test, label_train, label_test = train_test_split(data, label, test_size=0.20, 
                                                                  random_state=0, stratify=label)

In [6]:
df_data_raw_train = pd.DataFrame(list(zip(data_train, label_train)), columns=['text', 'label'])
df_data_raw_test = pd.DataFrame(list(zip(data_test, label_test)), columns=['text', 'label'])

In [7]:
print(df_data_raw_train.shape, df_data_raw_test.shape)

(1859, 2) (465, 2)


In [8]:
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = []
    # antonyms = []

    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
                
    return list(set(synonyms))

In [9]:
import re

def replace_synonym(sentence, word):
    
    augmented_sentences = []
    augmented_sentences.append(sentence)
    
    sentence_words = sentence.split(' ')
    
    if word in sentence_words:
        
        synonyms = get_synonyms(word)
    
        for synonym in synonyms:
            reg_ex = r'\b'+word+r'\b'
            new_setence = re.sub(reg_ex, synonym, sentence)
        
            augmented_sentences.append(new_setence)
        
    return augmented_sentences

In [14]:
def synonym_augmentation(sentence):
    
    sentences = []
    
    words = sentence.split(' ')
    
    for word in words:
        # print('---------------------\n', word)
        new_sentences = replace_synonym(sentence, word)
        # print(new_sentences)
        sentences.extend(new_sentences)
        
    return list(set(sentences))

In [15]:
def synonym_augmentation_withoutstopwords(sentence):
    
    sentences = []
    
    words = sentence.split(' ')
    
    for word in words:
        # print('---------------------\n', word)
        if word not in stopwords.words('english'):
            new_sentences = replace_synonym(sentence, word)
            # print(new_sentences)
            sentences.extend(new_sentences)
        
    return list(set(sentences))

In [12]:
df_imperative_train = df_data_raw_train.loc[df_data_raw_train['label'] == 1]
df_nonimperative_train = df_data_raw_train.loc[df_data_raw_train['label'] == 0]

# df_imperative_test = df_data_raw_test.loc[df_data_raw_test['label'] == 1]
# df_nonimperative_test = df_data_raw_test.loc[df_data_raw_test['label'] == 0]

In [16]:
def get_augmented_dataframe(sentence, label):
    
    augmented_sentences = synonym_augmentation_withoutstopwords(sentence)
    labels = [label] * len(augmented_sentences)
    
    df = pd.DataFrame(list(zip(augmented_sentences, labels)), columns=['text', 'label'])
    
    return df

In [17]:
df_aug_imperative_train = pd.DataFrame(columns=['text', 'label'])
for index, row in df_imperative_train.iterrows():
    
    sentence = row['text']
    label = row['label']
    
    df = get_augmented_dataframe(sentence, label)
    
    df_aug_imperative_train = pd.concat([df_aug_imperative_train, df], axis=0, sort=False)

In [18]:
df_aug_imperative_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53760 entries, 0 to 171
Data columns (total 2 columns):
text     53760 non-null object
label    53760 non-null object
dtypes: object(2)
memory usage: 1.2+ MB


In [20]:
Counter(df_aug_imperative_train['label'])

Counter({1: 53760})

In [21]:
df_aug_imperative_train.head()

,text,label
0,Learn to enjoy cook from scratch,1
1,read to enjoy cooking from scratch,1
2,discover to enjoy cooking from scratch,1
3,Learn to enjoy cooking from incision,1
4,con to enjoy cooking from scratch,1


In [22]:
df_aug_nonimperative_train = pd.DataFrame(columns=['text', 'label'])
for index, row in df_nonimperative_train.iterrows():
    
    sentence = row['text']
    label = row['label']
    
    df = get_augmented_dataframe(sentence, label)
    
    df_aug_nonimperative_train = pd.concat([df_aug_nonimperative_train, df], axis=0, sort=False)

In [23]:
Counter(df_aug_nonimperative_train['label'])

Counter({0: 46362})

In [24]:
df_aug_nonimperative_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46362 entries, 0 to 442
Data columns (total 2 columns):
text     46362 non-null object
label    46362 non-null object
dtypes: object(2)
memory usage: 1.1+ MB


In [25]:
df_aug_nonimperative_train.head()

,text,label
0,Gopher snakes in Arizona are not poisonous bu...,0
1,Gopher snakes in Arizona are not poisonous bu...,0
2,Gopher snakes in Arizona are not poisonous bu...,0
3,Gopher snakes in Arizona are not vicious but ...,0
4,Gopher snakes in Arizona are not poisonous bu...,0


In [26]:
df_data_train = pd.concat([df_aug_nonimperative_train, df_aug_imperative_train], axis=0, sort=False)

In [27]:
ip_file = os.path.join(INPUT_DIR, 'imperatives_binary_synonym_aug_train.csv')
df_data_train.to_csv(ip_file, index=False)

In [28]:
Counter(df_data_train['label'])

Counter({0: 46362, 1: 53760})

In [29]:
data_train = df_data_train['text']
label_train = df_data_train['label']

data_test = df_data_raw_test['text']
label_test = df_data_raw_test['label']

In [30]:
from collections import Counter

print('Training Data split', Counter(label_train))
print('Testing Data split', Counter(label_test))

Training Data split Counter({1: 53760, 0: 46362})
Testing Data split Counter({1: 234, 0: 231})


In [31]:
df_data_train = pd.DataFrame(list(zip(data_train, label_train)), columns=['text', 'label'])
df_data_test = pd.DataFrame(list(zip(data_test, label_test)), columns=['text', 'label'])

In [32]:
df_data_test.head()

,text,label
0,and my first paper I got a C on and I'd knew t...,0
1,Shut the door,1
2,I took some summer courses.,0
3,You better be quit!,1
4,So.,0


## Preprocess data

In [33]:
import re

def preprocess_text(text):
    
    # print(text, end ='')
    
    text = text.lower()
    text = re.sub(r'\W',' ', text)
    text = re.sub(' \d+', ' ', text)
    text = re.sub(r'\s+',' ', text)
        
    words = text.split(' ')
    words = [w.strip() for w in words]
    
    text = ' '.join(words)
    text = text.strip()
        
    # print(text)
    return text

In [34]:
def preprocess_df(df_data):
    df_data['processed_text'] = df_data['text'].apply(lambda x: preprocess_text(x))
    df_data.drop(df_data[df_data['processed_text'] == ''].index, inplace=True)
    
    return df_data

In [35]:
# Preprocess train & test data frame
print('Train before cleaning', df_data_train.shape)
df_data_train = preprocess_df(df_data_train)
print('Train after cleaning', df_data_train.shape)


print('Test before cleaning', df_data_test.shape)
df_data_test = preprocess_df(df_data_test)
print('Test after cleaning', df_data_test.shape)

Train before cleaning (100122, 2)
Train after cleaning (100122, 3)
Test before cleaning (465, 2)
Test after cleaning (465, 3)


In [36]:
df_data_test.head()

,text,label,processed_text
0,and my first paper I got a C on and I'd knew t...,0,and my first paper i got a c on and i d knew t...
1,Shut the door,1,shut the door
2,I took some summer courses.,0,i took some summer courses
3,You better be quit!,1,you better be quit
4,So.,0,so


### Create Corpus using only train data

In [37]:
corpus = df_data_train['text'].values
print('Corpus Length ', len(corpus))

Corpus Length  100122


### Vectorization

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

## Use Bag of Words Vectorizer for encoding`
vectorizer = CountVectorizer()
vectorizer.fit(corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

### Vectorization of Train

In [39]:
data_train = vectorizer.transform(df_data_train['text'])
print('Shape of the data train:',data_train.shape)

Shape of the data train: (100122, 14035)


### Train

In [40]:
label_train = np.array(df_data_train['label'])
label_train = label_train.reshape((len(label_train), 1))
label_train

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

### Test


In [41]:
data_test = vectorizer.transform(df_data_test['text'])
print('Shape of the data train:',data_test.shape)

Shape of the data train: (465, 14035)


In [42]:
label_test = np.array(df_data_test['label'])
label_test = label_test.reshape((len(label_test), 1))

# Estimator

## SVM with poly (3) kernel - with stopwords

In [51]:
from sklearn.svm import SVC

# Run Logistic Regression
estimator = SVC(kernel='poly', degree=3)
estimator.fit(data_train, label_train)
predictions = estimator.predict(data_test)

predictions

/Users/talat/anaconda3/envs/sentgen/lib/python3.5/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,

In [52]:
from sklearn.metrics import f1_score


## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, label_train))
print('Test Accuracy', estimator.score(data_test, label_test))

# F1
f1_measure = f1_score(label_test, predictions, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(label_test, predictions))

Train Accuracy 0.9510519570460824
Test Accuracy 0.632258064516129
F1 macro Score:  0.5920403880785392
              precision    recall  f1-score   support

           0       0.84      0.32      0.46       231
           1       0.58      0.94      0.72       234

    accuracy                           0.63       465
   macro avg       0.71      0.63      0.59       465
weighted avg       0.71      0.63      0.59       465



In [53]:
model_file = MODEL_DIR + 'svm/' + 'model_svm_poly3_synonym_aug_withstopwords.pkl'
with open(model_file, 'wb') as f_op:
    pkl.dump(estimator, f_op)

## SVM with linear kernel - with stopwords

In [43]:
from sklearn.svm import SVC

estimator = SVC(kernel='linear')
estimator.fit(data_train, label_train)
predictions = estimator.predict(data_test)

predictions

/Users/talat/anaconda3/envs/sentgen/lib/python3.5/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,

In [44]:
from sklearn.metrics import f1_score


## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, label_train))
print('Test Accuracy', estimator.score(data_test, label_test))

# F1
f1_measure = f1_score(label_test, predictions, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(label_test, predictions))

Train Accuracy 0.9971035336888996
Test Accuracy 0.789247311827957
F1 macro Score:  0.7883064516129032
              precision    recall  f1-score   support

           0       0.83      0.73      0.77       231
           1       0.76      0.85      0.80       234

    accuracy                           0.79       465
   macro avg       0.79      0.79      0.79       465
weighted avg       0.79      0.79      0.79       465



In [45]:
model_file = MODEL_DIR + 'svm/' + 'model_svm_linear_synonym_aug_withstopwords.pkl'
with open(model_file, 'wb') as f_op:
    pkl.dump(estimator, f_op)

## Deep Neural Network Model

In [46]:
from keras.models import Sequential
from keras import layers

In [47]:
input_dim = data_train.shape[1]  # Number of features

In [49]:
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                140360    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 140,371
Trainable params: 140,371
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(data_train, label_train,
                    epochs=100,
                    verbose=True, 
                    validation_data=(data_test, label_test),
                    batch_size=10)

Epoch 1/100
10013/10013 [==============================] - 8s 775us/step - loss: 0.0029 - accuracy: 0.9991 - val_accuracy: 0.8172 - val_loss: 2.2232
Epoch 2/100
10013/10013 [==============================] - 8s 783us/step - loss: 0.0022 - accuracy: 0.9993 - val_accuracy: 0.8215 - val_loss: 2.3012
Epoch 3/100
10013/10013 [==============================] - 8s 775us/step - loss: 0.0017 - accuracy: 0.9994 - val_accuracy: 0.8043 - val_loss: 2.4633
Epoch 4/100
10013/10013 [==============================] - 7s 738us/step - loss: 0.0014 - accuracy: 0.9995 - val_accuracy: 0.8172 - val_loss: 2.6541
Epoch 5/100
10013/10013 [==============================] - 8s 796us/step - loss: 0.0012 - accuracy: 0.9996 - val_accuracy: 0.8043 - val_loss: 2.6704
Epoch 6/100
10013/10013 [==============================] - 8s 835us/step - loss: 9.2613e-04 - accuracy: 0.9996 - val_accuracy: 0.8108 - val_loss: 2.8776
Epoch 7/100
10013/10013 [==============================] - 8s 780us/step - loss: 8.1199e-04 - accuracy

In [ ]:
loss, accuracy = model.evaluate(data_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(data_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()